In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, random_split
import os
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [2]:
labels = ['PNEUMONIA', 'NORMAL']
dataset = []
test_DS = []
val_DS = []

# Populate the dataset list for training

for label in labels:
    path = os.path.join("./Dataset/chest_xray/train", label)
    class_num = labels.index(label)
    for img_name in os.listdir(path):
        dataset.append({"img_name": os.path.join(label, img_name), "label": class_num})

# Populate the dataset list for testing

for label in labels:
    path = os.path.join("./Dataset/chest_xray/test", label)
    class_num = labels.index(label)
    for img_name in os.listdir(path):
        test_DS.append({"img_name": os.path.join(label, img_name), "label": class_num})

# Populate the dataset list for validation

for label in labels:
    path = os.path.join("./Dataset/chest_xray/val", label)
    class_num = labels.index(label)
    for img_name in os.listdir(path):
        val_DS.append({"img_name": os.path.join(label, img_name), "label": class_num})


In [3]:
train_df = pd.DataFrame(dataset)
test_df = pd.DataFrame(test_DS)
val_df = pd.DataFrame(val_DS)


In [4]:
img_size = 150
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor()
])

In [5]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.dataframe = annotation_file
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, index):
        img_name = os.path.join(self.root_dir, self.dataframe.iloc[index, 0])
        image = Image.open(img_name).convert("RGB")
        label = int(self.dataframe.iloc[index, 1])

        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
train_dataset = CustomDataset(root_dir="./Dataset/chest_xray/train", annotation_file=train_df, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = CustomDataset(root_dir="./Dataset/chest_xray/test", annotation_file=test_df, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

val_dataset = CustomDataset(root_dir="./Dataset/chest_xray/val", annotation_file=val_df, transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [7]:
class ConvNeuralNetwork(nn.Module):
    def __init__(self, num_classes):
        super(ConvNeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, padding=2)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=7, padding=3)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=11, padding=4)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self._initialize_fc_input_size(img_size)
        self.fc1 = nn.Linear(self.fc_input_size, 128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def _initialize_fc_input_size(self, img_size):
        # Dummy forward pass to calculate input size for fully connected layer
        x = torch.randn(1, 1, img_size, img_size)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.maxpool1(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.maxpool2(x)
        self.fc_input_size = x.view(x.size(0), -1).size(1)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.maxpool1(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.maxpool2(out)
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

In [8]:
batch_size = 32
num_classes = 2
learning_rate = 0.001
num_epochs = 2
model = ConvNeuralNetwork(num_classes)
model.load_state_dict(torch.load("lungs.pth"))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.005, momentum=0.9)


C:\Users\iamgo\AppData\Local\Temp\ipykernel_17592\340878123.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("lungs.pth"))


In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    n=0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        y_pred = model(inputs)
        loss = criterion(y_pred, labels)
        n=n+1
        print(f"{n}-loss:{loss.item():.4f}")
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    model.eval()
    with torch.no_grad():
        test_acc = 0
        count = 0
        for inputs, labels in test_loader:
            y_pred = model(inputs)
            test_acc += (torch.argmax(y_pred, 1) == labels).float().sum()
            count += len(labels)
        test_acc /= count

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}, Test Accuracy: {test_acc*100:.2f}%")


In [ ]:
model.eval()
val_acc = 0
val_count = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        y_pred = model(inputs)
        
        # Get predicted and actual labels
        predicted_labels = torch.argmax(y_pred, 1)
        
        # Print predicted and actual values
        for pred, actual in zip(predicted_labels, labels):
            print(f"Predicted: {pred.item()}, Actual: {actual.item()}")
        
        val_acc += (predicted_labels == labels).float().sum()
        val_count += len(labels)
val_acc /= val_count
print(f"Validation Accuracy: {val_acc*100:.2f}%")

In [ ]:
# Save model
torch.save(model.state_dict(), "lungs.pth")

In [9]:
import torch
img_s=(150,150)
bat_size=1
onnx_model_path='lungs.onnx'
model.eval()
sample_inp=torch.rand((bat_size,1,*img_s))

y=model(sample_inp)
torch.onnx.export(
    model,
    sample_inp,
    onnx_model_path,
    verbose=False,
    input_names=['input'],
    output_names=['output'],
    opset_version=12
)
print(f"Model has been converted to ONNX format and saved at {onnx_model_path}")

Model has been converted to ONNX format and saved at lungs.onnx


In [11]:
import onnx
from onnx_tf.backend import prepare


In [12]:
onnx_model_path='lungs.onnx'
tf_model_path='model_tf'
onnx_model=onnx.load(onnx_model_path)
tf_rep=prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

INFO:absl:Function `__call__` contains input name(s) x with unsupported characters which will be renamed to transpose_17_x in the SavedModel.
INFO:absl:Found untraced functions such as gen_tensor_dict while saving (showing 1 of 1). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: model_tf\assets


INFO:tensorflow:Assets written to: model_tf\assets
INFO:absl:Writing fingerprint to model_tf\fingerprint.pb


In [13]:
import tensorflow as tf
saved_model_dir='model_tf'
tflite_model_path='model.tflite'

In [14]:
converter=tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model=converter.convert()

In [15]:
with open(tflite_model_path,'wb') as f:
    f.write(tflite_model)

In [17]:
interpreter=tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

input_details=interpreter.get_input_details()
output_details=interpreter.get_output_details()

input_shape=input_details[0]['shape']
input_data=np.array(np.random.random_sample(input_shape),dtype=np.float32)

interpreter.set_tensor(input_details[0]['index'],input_data)

interpreter.invoke()
output_data =interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[ 0.7542044 -0.6305524]]


In [ ]:
converter=tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite

In [12]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [13]:
%pip install matplotlib

In [14]:
%pip install onnx


Note: you may need to restart the kernel to use updated packages.


In [15]:
%pip install tensorflow-addons


  Using cached typeguard-2.13.3-py3-none-any.whl.metadata (3.6 kB)
   ---------------------------------------- 0.0/719.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/719.8 kB ? eta -:--:--
    --------------------------------------- 10.2/719.8 kB ? eta -:--:--
    --------------------------------------- 10.2/719.8 kB ? eta -:--:--
    --------------------------------------- 10.2/719.8 kB ? eta -:--:--
   - ------------------------------------- 30.7/719.8 kB 131.3 kB/s eta 0:00:06
   - ------------------------------------- 30.7/719.8 kB 131.3 kB/s eta 0:00:06
   - ------------------------------------- 30.7/719.8 kB 131.3 kB/s eta 0:00:06
   -- ------------------------------------ 41.0/719.8 kB 115.9 kB/s eta 0:00:06
   --- ----------------------------------- 61.4/719.8 kB 156.1 kB/s eta 0:00:05
   ---- ---------------------------------- 81.9/719.8 kB 199.7 kB/s eta 0:00:04
   ---- ---------------------------------- 92.2/719.8 kB 210.1 kB/s eta 0:00:03
   ----- -----

In [24]:
!git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow && pip install -e .

Obtaining file:///D:/ML/Lung%20Cancer/onnx-tensorflow
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.9 MB 660.6 kB/s eta 0:00:11
    --------------------------------------- 0.1/6.9 MB 980.4 kB/s eta 0:00:07
    --------------------------------------- 0.2/6.9 MB 984.6 kB/s eta 0:00:07
   - -------------------------------------- 0.2/6.9 MB 1.1 MB/s eta 0:00:07
   - -------------------------------------- 0.2/6.9 MB 1.0 MB/s eta 0:00:07
   - -------------------------------------- 0.3/6.9 MB 1.1 MB/s eta 0:00:06
   -- ------------------------------------- 0.4/6.9 MB 1.2 MB/s eta 0:00:06
   -- ------------------------------------- 0.4/6.9 MB 1.1 MB/s eta 0:00:06
   -- ----------------------

Cloning into 'onnx-tensorflow'...


In [18]:
%pip install torch

Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [20]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [25]:
%pip install onnx_tf

Note: you may need to restart the kernel to use updated packages.
